### Stock Market Prediction And Forecasting Using Stacked LSTM

In [ ]:
### Keras and Tensorflow >2.0

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('Best_accuaracy_data.csv')

In [4]:
df.head()

,Unnamed: 0,skipped,session_length,short_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,duration,release_year,us_popularity_estimate,beat_strength,...,start_backbtn,start_clickrow,start_endplay,start_fwdbtn,start_trackdone,end_backbtn,end_clickrow,end_endplay,end_fwdbtn,end_trackdone
0,0,0.0,20.0,0.0,0.0,0.0,180.066666,2018.0,99.968133,0.438551,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,1,0.0,20.0,0.0,0.0,0.0,236.796371,2018.0,99.896728,0.654804,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,2,0.0,20.0,0.0,0.0,0.0,231.266663,2018.0,99.999976,0.532155,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,3,0.0,20.0,0.0,0.0,0.0,169.826675,2018.0,99.995038,0.641756,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,4,0.0,20.0,0.0,0.0,0.0,210.545258,2018.0,99.998498,0.732428,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [5]:
df.tail()

,Unnamed: 0,skipped,session_length,short_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,duration,release_year,us_popularity_estimate,beat_strength,...,start_backbtn,start_clickrow,start_endplay,start_fwdbtn,start_trackdone,end_backbtn,end_clickrow,end_endplay,end_fwdbtn,end_trackdone
106216,106216,1.0,20.0,0.0,0.0,0.0,167.213333,2014.0,99.796168,0.384028,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
106217,106217,1.0,20.0,0.0,0.0,0.0,248.413330,2014.0,99.799035,0.529222,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
106218,106218,1.0,20.0,0.0,0.0,0.0,214.693329,2014.0,99.775823,0.420086,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
106219,106219,1.0,20.0,0.0,0.0,0.0,270.506683,2014.0,99.783735,0.371785,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
106220,106220,0.0,10.0,0.0,0.0,2.0,1.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df1=df.reset_index()['skipped']

In [7]:
df1

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
106216    1.0
106217    1.0
106218    1.0
106219    1.0
106220    0.0
Name: skipped, Length: 106221, dtype: float64

In [9]:
### LSTM are sensitive to the scale of the data. so we apply MinMax scaler 

In [10]:
import numpy as np

In [11]:
df1

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
106216    1.0
106217    1.0
106218    1.0
106219    1.0
106220    0.0
Name: skipped, Length: 106221, dtype: float64

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

In [13]:
print(df1)

[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [1.]
 [0.]]


In [14]:
##splitting dataset into train and test split
training_size=int(len(df1)*0.65)
test_size=len(df1)-training_size
train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]

In [15]:
training_size,test_size

(69043, 37178)

In [16]:
train_data

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]])

In [17]:
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [18]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [19]:
print(X_train.shape), print(y_train.shape)

(68942, 100)
(68942,)


(None, None)

In [20]:
print(X_test.shape), print(ytest.shape)

(37077, 100)
(37077,)


(None, None)

In [21]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [22]:
### Create the Stacked LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [23]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')


In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 50)           10400     
                                                                 
 lstm_1 (LSTM)               (None, 100, 50)           20200     
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 50)           10400     
                                                                 
 lstm_1 (LSTM)               (None, 100, 50)           20200     
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=1,batch_size=64,verbose=1)

1078/1078 [==============================] - 222s 206ms/step - loss: 0.1839 - val_loss: 0.1829


In [28]:
import tensorflow as tf

In [29]:
tf.__version__

'2.8.2'

In [30]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [31]:
##Transformback to original form
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [32]:
### Calculate RMSE performance metrics
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(y_train,train_predict))

0.4279110586356475

In [41]:
### Test Data RMSE
math.sqrt(mean_squared_error(ytest,test_predict))

0.42767610706369724